### Introduction

This notebook demonstrate exploring detailed map matching information generated from FMM.

Packages used:

- fmm
- shapely
- ipyleaflet
- ipywidgets
- bqplot
- pandas
- numpy



### Load FMM model

In [1]:
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig
from shapely.geometry import LineString,shape, mapping
import json
from ipyleaflet import GeoJSON
from shapely.wkt import dumps, loads
import pandas as pd
from __future__ import print_function

In [2]:
network = Network("data/stockholm/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)
# # Skip the precomputation if you already generate that file
# ubodt_gen = UBODTGenAlgorithm(network,graph)
# status = ubodt_gen.generate_ubodt("data/stockholm/ubodt.txt", 0.03, binary=False, use_omp=True)
# print status
ubodt = UBODT.read_ubodt_csv("data/stockholm/ubodt.txt")
### Create FMM model
fmm_model = FastMapMatch(network,graph,ubodt)

Nodes 6974 edges 15021


In [3]:
traj_style = {
  'opacity':0.8,
  'weight':5,
  'color':'orange',
  'dashArray': '9'
}
traj_point_style={
    'radius': 8, 
    'color': 'orange', 
    'fillOpacity': 1.0, 
    'fillColor': 'orange', 
    'weight': 2
}
mr_style = {
    'color':'#3388ff',
   'opacity':1.0,
   'weight':10
}
traj_point_hover_style = {
   'fillColor': 'white',
   'color':'#9932CC',
   'weight':5,
   'radius': 10 ,
    'fillOpacity':1.0
}
def extract_detailed_match_info(result):
    if (len(list(result.cpath))>0):
        candidates = []
        for c in result.candidates:
            candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))
        df = pd.DataFrame(candidates,
            columns=["eid","source","target","error","length","offset","spdist","ep","tp"])
        df["error"] = df["error"]*1.1e5
        df["length"] = df["length"]*1.1e5
        df["offset"] = df["offset"]*1.1e5
        df["spdist"] = df["spdist"]*1.1e5
        return df
    else:
        return None
    
def match_geojson_network_details(traj, k, radius, gps_error):
    traj_wkt = shape(traj["geometry"]).wkt
    r_degree = radius/1.1e5
    e_degree = gps_error/1.1e5
    fmm_config = FastMapMatchConfig(k,r_degree,e_degree)
    result = fmm_model.match_wkt(traj_wkt, fmm_config)
    mr_wkt = result.mgeom.export_wkt()
    if len(list(result.cpath))>0:
        return GeoJSON(name="Matched Traj",
                          data=mapping(loads(mr_wkt)), 
                           style = mr_style), extract_detailed_match_info(result)
    else:
        return None, None
from shapely.geometry import Point
def traj2points(traj_geojson):
    geojson = {"type": "FeatureCollection",
            "features": [] }
    for index,c in enumerate(traj_geojson["geometry"]["coordinates"]):
        geojson["features"].append(
            {'type': 'Feature', 'properties': {"index":index}, 'geometry': mapping(Point(c[0],c[1]))})
    return geojson    
def traj_geojson_layer(traj):
    return GeoJSON(name="Traj",
                      data=traj, 
                       style = traj_style)

def traj_point_layer(traj_pts):
    return GeoJSON(name="Point",
                      data=traj_pts, 
                       point_style = traj_point_style, hover_style = traj_point_hover_style)

### Define a trajectory in GeoJSON

In [4]:
example_geojson = {u'geometry': 
{u'coordinates': [
   [18.041878, 59.340045],
   [18.038737, 59.346067],
   [18.04301, 59.350718],
   [18.050444, 59.348913],
   [18.053103, 59.351853],
   [18.062748, 59.350165],
   [18.062373, 59.347778],
   [18.071155, 59.346734],
   [18.070762, 59.343549],
   [18.070412, 59.339287],
   [18.06508, 59.338175],
   [18.05766, 59.335946]],
  u'type': u'LineString'},
 u'properties': {u'style': {u'clickable': True,
   u'color': u'#fca45d',
   u'fill': False,
   u'opacity': 1,
   u'stroke': True,
   u'weight': 6}},
 u'type': u'Feature'}


### Run map matching and visualize the result

In [5]:
import numpy as np 
from bqplot import LinearScale, Lines, Axis, Figure, ColorScale, Scatter
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker
from ipywidgets import Layout,HBox,Label,VBox, Dropdown, Output

In [6]:
l = Layout(flex="l",height="540px",width="50%")
m = Map(center=(59.34316546768387,18.04951351135969), zoom=14, 
        basemap=basemaps.OpenStreetMap.Mapnik,layout=l)
traj_layer = traj_geojson_layer(example_geojson)
mr_layer, details = match_geojson_network_details(example_geojson,
                32, 500, 200)
traj_p_layer= traj_point_layer(traj2points(example_geojson))


N = details.shape[0]
X = np.arange(N)

sc_x = LinearScale()
sc_y = LinearScale()

attributes = ["error","length","offset","spdist","ep","tp"]
descriptions = ["GPS error (meter)","Edge length (meter)",
                "Offset (meter)","SP length (meter)",
                "Emission Proability","Transit Probability"]
default_attribute = "error"

data_point = Scatter(x=X, y=details[default_attribute].values, scales={'x': sc_x, 'y': sc_y},
                     default_size=200,colors=["orange"])
data_point.hovered_style={             
    "fill":"white",
    "stroke":"#9932CC",
    "stroke-width":5
}
dropdown_options=[]
for d,v in zip(descriptions,attributes):
    dropdown_options.append((d,v))

w = Dropdown(
    options=dropdown_options,
    value=default_attribute,
    description='Feature:',
    layout=Layout(
         width="50%"   
    )
)

ax_x = Axis(scale=sc_x, label='Point index')
ax_y = Axis(scale=sc_y, orientation='vertical', label=w.label)

label = Label()
label2 = Label()


    
def on_dropdown_change(change):
    data_point.y = details[w.value].values
    ax_y.label = w.label
    label.value = w.value
    label2.value = "" 
    
hovered_marker = CircleMarker()
hovered_marker.fill_color = traj_point_hover_style["fillColor"]
hovered_marker.color = traj_point_hover_style["color"]
hovered_marker.weight = traj_point_hover_style["weight"]
hovered_marker.radius = traj_point_hover_style["radius"]
hovered_marker.fill_opacity = traj_point_hover_style["fillOpacity"]

hover_state = {
    "layer":hovered_marker,
    "onMap":False,
    "id":None
}

def data_point_on_hover(self, target):
    global hover_state
    hover_state["id"] = target["data"]["index"]
    label2.value = "{:.4f}".format(details[w.value].values[target["data"]["index"]])
    xy = traj_p_layer.data["features"][hover_state["id"]]["geometry"]["coordinates"]
    hover_state["layer"].location = (xy[1],xy[0]) 
    m.add_layer(hover_state["layer"])
    hover_state["onMap"] = True

def data_point_on_observe(state):
    global hover_state
    if (data_point.hovered_point == None):
        label2.value = ""
        if hover_state["onMap"]:
            m.remove_layer(hover_state["layer"])
            hover_state["onMap"]=False
            hover_state["id"]=None
    
data_point.on_hover(data_point_on_hover)
data_point.observe(data_point_on_observe)    
w.observe(on_dropdown_change)

fig = Figure(marks=[data_point], axes=[ax_x, ax_y],height="100%")    

w.observe(on_dropdown_change)

m.add_layer(traj_layer)
m.add_layer(mr_layer)
m.add_layer(traj_p_layer)

def map_on_mouseout(**kargws):
    data_point.hovered_point = None
    label2.value = ""
    hover_state["id"]=None
    if hover_state['onMap']:
        m.remove_layer(hovered_marker)
        hover_state['onMap']=False
    
def map_on_hover(event, feature, **kwargs):
    hover_state["id"] = feature["properties"]["index"]
    data_point.hovered_point = feature["properties"]["index"]
    label2.value = "{:.4f}".format(details[w.value].values[feature["properties"]["index"]])
# # traj_p_layer.on_mouseout(p_on_mouseout)
traj_p_layer.on_hover(map_on_hover)
traj_p_layer.on_mouseout(map_on_mouseout)
# print(data_point.hovered_point)
#VBox([HBox([label,label2]),HBox([m,VBox([w,fig],layout=l)])])
HBox([m,VBox([HBox([w,label2]),fig],layout=l)])

SEJveChjaGlsZHJlbj0oTWFwKGNlbnRlcj1bNTkuMzQzMTY1NDY3NjgzODcsIDE4LjA0OTUxMzUxMTM1OTY5XSwgY29udHJvbHM9KFpvb21Db250cm9sKG9wdGlvbnM9W3UncG9zaXRpb24nLCDigKY=
